In [46]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
import xvec
import os

## Constants Set Up

In [47]:
mtbs_raster_data_dir = "./data/mtbs_rasters"
mtbs_perimeter_data_path = "./data/mtbs_perimeters/mtbs_perims_DD.shp"
eia_utility_service_areas_path = "./data/eia_service_areas/Electric_Retail_Service_Territories.geojson"
fwi_zarr_path = "https://portal.nccs.nasa.gov/datashare/GlobalFWI/v2.0/fwiCalcs.GEOS-5/Default/GPM.LATE.v5/zarr/"
fwi_zarr_variable = "GPM.LATE.v5_FWI"

CRS = "EPSG:4269"

## Step 1)

First, we read in the MTBS Wildfire data files and package them in a list of dictionaries



In [ ]:
mtbs_raster_files = os.listdir(mtbs_raster_data_dir)

mtbs_data = []

mtbs_gdf = gpd.read_file(mtbs_perimeter_data_path)

ds_fwi = xr.open_zarr(fwi_zarr_path)
ds_fwi = ds_fwi.rio.write_crs("EPSG:4326")

ds_fwi = ds_fwi.rename({"lon": "x", "lat": "y"})

for file in mtbs_raster_files:
    data_struct = {}

    file_info = file[:-4].split("_")
    state = file_info[1]
    year = int(file_info[2])

    data_struct["state"] = state
    data_struct["year"] = year

    da_mtsb_raster = rioxarray.open_rasterio(f"{mtbs_raster_data_dir}/{file}")
    da_mtsb_raster = da_mtsb_raster.rio.reproject(CRS)
    # We filter here to include only values of meaningful fire severity
    da_mtsb_raster = da_mtsb_raster.where((da_mtsb_raster >= 2) & (da_mtsb_raster < 5))
    data_struct["da_fire_severity"] = da_mtsb_raster.copy()

    gdf_filtered = (mtbs_gdf.loc[(mtbs_gdf["Incid_Type"]=="Wildfire") & (mtbs_gdf["Ig_Date"].dt.year == data_struct["year"]) & (mtbs_gdf["Event_ID"].str.startswith(data_struct["state"]))])
    gdf_temp = gdf_filtered.set_index("Event_ID")[["geometry"]]
    gdf_temp = gdf_temp.to_crs(da_mtsb_raster.spatial_ref.crs_wkt)
    data_struct["gdf"] = gdf_temp.copy()

    ds_fwi_state = ds_fwi[fwi_zarr_variable].sel(time=ds_fwi.time.isin(gdf_filtered["Ig_Date"])).compute()
    ds_fwi_reproj = ds_fwi.rio.reproject_match(da_mtsb_raster)
    bounds = da_mtsb_raster.rio.bounds()
    ds_fwi_reproj_clipped = ds_fwi_reproj.rio.clip_box(*bounds)
    data_struct["da_fwi"] = ds_fwi_reproj_clipped.copy()

    print(f"{state} {year} processed!")
    mtbs_data.append(data_struct)


ClientResponseError: 503, message='Service Unavailable', url='https://portal.nccs.nasa.gov/datashare/GlobalFWI/v2.0/fwiCalcs.GEOS-5/Default/GPM.LATE.v5/zarr/GPM.LATE.v5_BUI/7.12.19'

In [ ]:
import geopandas as gpd
import xarray as xr
import rioxarray
import xvec

In [ ]:
calcd_dfs = []
da_name = "fire_severity"

for data in mtbs_data:

    da_temp = data["da"].xvec.zonal_stats(
        data["gdf"].geometry,
        x_coords="x",
        y_coords="y",
        stats=["mean", "max"],
        method="exactextract",
        index=True,
    )
    da_temp.name = da_name
    df_temp = da_temp.to_dataframe().reset_index()

    df_mean = df_temp.loc[df_temp["zonal_statistics"]=="mean"]
    df_mean = df_mean.rename(columns={da_name: da_name+"_mean"})

    df_max = df_temp.loc[df_temp["zonal_statistics"]=="max"]
    df_max = df_max.rename(columns={da_name: da_name+"_max"})


    df_temp =df_mean.merge(df_max, how="inner", on="Event_ID")[["Event_ID", da_name+"_mean", da_name+"_max"]]
    calcd_dfs.append(df_temp.copy())

    state = data["state"]
    year = data["year"]
    print(f"{state} {year} processed!")

fire_severity_df = pd.concat(calcd_dfs)
fire_severity_df = fire_severity_df.drop_duplicates(subset=["Event_ID"])
mtbs_gdf_merged = mtbs_gdf.merge(fire_severity_df, how='left', on="Event_ID")


In [ ]:
service_areas = gpd.read_file(eia_utility_service_areas_path)
service_areas = service_areas.to_crs(CRS)

service_areas = service_areas[["NAME", "geometry"]]
mtbs_gdf_final = mtbs_gdf_merged[["Event_ID", "Ig_Date", "BurnBndAc", "fire_severity_mean", "fire_severity_max", "geometry"]]

df_final = service_areas.sjoin(mtbs_gdf_final)
df_final = df_final.drop(columns=["geometry", "index_right"])
df_final = df_final.dropna(subset="fire_severity_mean")

In [ ]:
df_final["Ig_Date"]

In [ ]:
service_areas[["NAME"]].to_csv("service_area_names.csv", index=False)

In [ ]:
fire_severity_df

In [ ]:
ds_test.rio.write_crs("EPSG:4326", inplace=True)
ds_test.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
ds_test.rio.reproject(CRS)

In [ ]:
data["da"]